<h1 style="background-color: green;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐תהליך הסבת חלקות (דו-ממדיות) בשטח לא-מוסדר לבנק"ל מודרני🌐

In [ ]:
print('14. Unregistered Parcels Borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import datetime as dt
import pandas as pd
import geopandas as gpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
RawData_Folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData/'

today = dt.date.today()
product_name = str(today.year) + str(today.month) + str(today.day) + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + product_name
del [today, product_name]

In [ ]:
parcel_all_cols = ['PARCEL_ID', 'GUSH_NUM', 'GUSH_SUFFIX', 'PARCEL', 'STATUS', 'LEGAL_AREA', 'TALAR_NUMBER', 'TALAR_YEAR', 'geometry']

parcel_all = gpd.read_file(RawData_Folder + 'ArcStorm\PARCEL_ALL.gdb',
                           layer          = 'PARCEL_ALL',
                           include_fields = parcel_all_cols)


parcel_all = parcel_all.to_crs(2039)

del [parcel_all_cols]

print('         input 1/3 read')

In [ ]:
cpb_cols = ['CPBUniqueID', 'ProcessName', 'ProcessType']

cpb_layer = gpd.read_file(product_path,
                          layer = 'CadasterProcessBorders',
                          include_fields = cpb_cols,
                          ignore_geometry = True)

unreg_tazars = cpb_layer[cpb_layer['ProcessType'] == 11]


del [cpb_cols, cpb_layer]

print('         input 2/3 read')

In [ ]:
Blocks_cols = ['BlockNumber', 'SubBlockNumber', 'BlockUniqueID']

Blocks = gpd.read_file(product_path,
                       layer = 'Blocks',
                       include_fields = Blocks_cols,
                       ignore_geometry = True)

Blocks = Blocks.astype(int)


del[Blocks_cols, RawData_Folder, product_path]

print('         input 3/3 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
registered_status = [1, 3, 6, 16]

unreg_parcels = parcel_all[~parcel_all['STATUS'].isin(registered_status)]
unreg_parcels.drop(columns='STATUS', inplace=True)

del registered_status, parcel_all

In [ ]:
unreg_parcels[['TALAR_NUMBER', 'TALAR_YEAR']] = unreg_parcels[['TALAR_NUMBER', 'TALAR_YEAR']].fillna(0)

unreg_parcels['ProcessName'] = unreg_parcels['TALAR_NUMBER'].astype(str) + '/' + unreg_parcels['TALAR_YEAR'].astype(str)
unreg_parcels['ProcessName'] = unreg_parcels['ProcessName'].replace('0/0', np.nan)

unreg_parcels.drop(columns=['TALAR_NUMBER', 'TALAR_YEAR'], inplace = True)

unreg_parcels = unreg_parcels.merge(unreg_tazars, on = 'ProcessName', how = 'left')

In [ ]:
unreg_parcels['GUSH_SUFFIX'] = unreg_parcels['GUSH_SUFFIX'].fillna(0)
unreg_parcels['GUSH_NUM'] = unreg_parcels['GUSH_NUM'].astype(int)
unreg_parcels['GUSH_SUFFIX'] = unreg_parcels['GUSH_SUFFIX'].astype(int)
unreg_parcels['BlockName'] = unreg_parcels['GUSH_NUM'].astype(str) + '_' + unreg_parcels['GUSH_SUFFIX'].astype(str)

Blocks['BlockName'] = Blocks['BlockNumber'].astype(str) + '_' + Blocks['SubBlockNumber'].astype(str)

unreg_parcels = unreg_parcels.merge(Blocks, on = 'BlockName', how = 'left')
unreg_parcels['BlockUniqueID'] = unreg_parcels['BlockUniqueID'].astype('Int64')
unreg_parcels.drop(columns=['GUSH_NUM', 'GUSH_SUFFIX', 'BlockName', 'BlockNumber', 'SubBlockNumber'], inplace = True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית